# CC471: LAB06 -2019 - Manejo de secuencias con BioPython


## Practica: 



# BioPython

BioPython es un paquete Python muy popular para manejar información biologica.

Para instalar:

```
conda install biopython
```
ó

```
pip install biopython
```


BioPython tiene tres funcionalidades principales:

* Sequence Handling
* 3D Structure
* Population Genetics


Referencias: [tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html), [website](http://biopython.org/), [wiki](http://biopython.org/wiki/Category%3AWiki_Documentation) (la mayoria de los ejemplos viene de estas fuentes).
 
# `BioPython.Seq`

El tipo de objeto principal que veremos son las secuencias. Se maneja con la clase `Seq`.

Como es sabido, podemos tener secuencias de tipo ADN , RNA, y  Proteinas .

BioPython ayuda a distinguir y hacer diferentes cosas con los diferentes tipos de secuencias.

In [2]:
#let's make a generic sequence

from Bio.Seq import Seq

my_seq = Seq("CCCGGAGAGA")
print(type(my_seq))
#let's see what attributes this object has
attributes = [a for a in dir(my_seq) if not a.startswith("_")]
print(attributes)

<class 'Bio.Seq.Seq'>
['alphabet', 'back_transcribe', 'complement', 'count', 'count_overlap', 'endswith', 'find', 'lower', 'lstrip', 'reverse_complement', 'rfind', 'rsplit', 'rstrip', 'split', 'startswith', 'strip', 'tomutable', 'tostring', 'transcribe', 'translate', 'ungap', 'upper']


Al parecer se puede hacer mucho con este tipo de objetos.

El problema es que Python no conoce que tipo de secencia es esta (DNA, RNA, Proteina).

debemos especificar a que tipo de "Alphabeto" pertenece a secuencia.

In [3]:
#right now it has just a generic alphabet
print(my_seq.alphabet)

Alphabet()


In [4]:
from Bio.Alphabet import generic_dna, generic_protein, generic_rna

my_dna = Seq("CCCGGAGAG", generic_dna)
my_rna = Seq("ACCCGUUGU", generic_rna)
my_protein = Seq("AKKKGGGUUULL", generic_protein)

Biopython ahora sabrá la diferencia entre una base de ADN `A` de adenina y un residuo de proteina `A` de  alanina.

ahora se puede realizar acciones del dogma central : transcripcion y traduccion.

In [5]:
my_gene = Seq("ACTAGCAGCGGA", generic_dna)

#get the mRNA

my_transcript = my_gene.transcribe()
print(my_transcript)
print(my_transcript.alphabet)


#get the protein from the mRNA
my_protein = my_transcript.translate()
print(my_protein)
print(my_protein.alphabet)

ACUAGCAGCGGA
RNAAlphabet()
TSSG
ExtendedIUPACProtein()


También se puede pasar directamente de un ADN a la proteina.

In [6]:
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG", generic_dna)
myprot = coding_dna.translate()
print(myprot)

MAIVMGR*KGAR*


Como se puede ver, tenemos algunos STOP codones representados como `*` y la traducción continuó.

podemos hacer que la traducción realmente pare cuando encuentre un STOP codon.

In [7]:
myprot = coding_dna.translate(to_stop=True)
print(myprot)

MAIVMGR


Incluso se puede ser mas realista, y permitir solamente la traducción de genes válidos  (por ejemplo,  con codones de START y STOP válidos y un numero apropiado de bases).

Esto se hace estableciendo  la keyword `cds=True` , que significa  "coding sequence".

Si no tuvieramos una coding secuence válida obtendremos un error.

In [8]:
myprot = coding_dna.translate(cds=True)

TranslationError: Extra in frame stop codon found.

In [10]:
gene = Seq("ATGGCCATTGTAATGTAG", generic_dna)
gene.translate(cds=True)

Seq('MAIVM', ExtendedIUPACProtein())

### Metodos generales para secuencias

Existen algunas operaciones convenientes para trabajar con secuencias.

se pueden concatenar sequencias si son de tipos identicos.

In [11]:
seq1 = Seq("AAACGGA", generic_dna)
seq2 = Seq("GGAGAT", generic_dna)

seq1 + seq2

Seq('AAACGGAGGAGAT', DNAAlphabet())

Tambien se puede indexar y cortar como si fueran  strings.

In [12]:
print(seq1[:5]) 
print(seq2[-2]) 
seq1[:5] + seq2[-2]

AAACG
A


Seq('AAACGA', DNAAlphabet())

los objetos tipo `Seq` son inmutables, como las strings.

In [13]:
seq1[2] = "G"

TypeError: 'Seq' object does not support item assignment

Existe otro tipo de objeto llamado  `MutableSeq`.  Si queremos utilizar mutabilidad podemos convertir un objeto `Seq` a otro objeto `MutableSeq` facilmente.

In [14]:
mut_seq = seq1.tomutable()
mut_seq

MutableSeq('AAACGGA', DNAAlphabet())

In [15]:
mut_seq[0] = "G"
print(mut_seq)

GAACGGA


Tambien podemos hacer búsquedas dentro de las secuencias

In [16]:
myseq = Seq("CCAGAAACCCGGAA", generic_dna)

#find the first occurence of the pattern
print(myseq.find("GAA"))

#find the number of non-overlapping occurrences of a pattern
print(myseq.count("GAA"))

3
2


# Busqueda en Bases de Datos

Digamos que ha aislado una parte de un ADN en el laboratorio y tiene su secuencia.

Ahora quiere encontrar a que organismo pertenece esta secuencia.

Si pregunta a un Biologo le dirá "BLASTealo ".

BLAST es un algoritmo de alineamiento que busca su secuencia en una enorme base de datos de secuencias cuyos origenes se conocen.

Sin la ayuda de BioPython, tendriamos que subir nuestras secuencias al servidor BLAST  `AAAAGGAGAGAGAGTTTATA` accediendo al servidor de NCBI [NCBI BLAST web server](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastn&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome). Que pasaria si tuvieramos cientos de secuencias para investigar?

Gracias a BioPython podemos hacerlo automaticamente!

El metodo  `qblast`de lo módulo `Bio.Blast.NCBIWWW` envia nuestra secuencia al sevidor  BLAST.

Aqui utilizaremos el el algoritmo de "BLAST para nucleotidos"  `blastn` y lo utilizaremos en la base de datos de todas las secuencias de nucleotidos, llamada `nt`.


In [31]:
from Bio.Blast import NCBIWWW

result_handle = NCBIWWW.qblast("blastn", "nt", Seq("ACCGCCGAGACCGCGTCCGCCCCGCGAGCACAGAGCCTCGCCTTTGCCGATCCGCCGCCCGTCCA", generic_dna))
print(result_handle)

###Esperamos algunos segundos 
para obtener un  `result_handle` que es como un archivo  temporal abierto para lectura.

El formato de este archivo es XML, Felizmente BioPython tiene un parser XML que extrae toda la información necesaria para nosotros.

In [32]:
from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(result_handle)
dir(blast_records)
print(blast_records.__sizeof__)

<built-in method __sizeof__ of generator object at 0x7f3eaf665b48>


Obtenemos un "iterator" de objetos tipo record de BLAST, o " resultados de la busqueda". Ahora se puede hacer un ciclo sobre cada uno de nuestros resultados  e imprimir alguna información.

Aqui hacemos un Loop sobre todos los resultados que tienen el atributo 
 `alignments` que son los alineamientos de nuestra secuencia con algun organismo en la base de datos.

El atributo `alignment` en si tiene otros atributos como la secuencia `query`, la longitud `length` y titulo `title` del organismo correspondiente

In [33]:
print('1')
for b in blast_records:
    for alignment in b.alignments:
        for hsp in alignment.hsps:
            print('****Alineamiento****')
            print('secuencia:', alignment.title)
            print('longitud:', alignment.length)
            print('e value:', hsp.expect)
            print(hsp.query[0:75] + '...')
            print(hsp.match[0:75] + '...')
            print(hsp.sbjct[0:75] + '...')

1
****Alineamiento****
secuencia: gi|1241781418|ref|NM_001101.4| Homo sapiens actin beta (ACTB), mRNA
longitud: 1940
e value: 1.29882e-23
ACCGCCGAGACCGCGTCCGCCCCGCGAGCACAGAGCCTCGCCTTTGCCGATCCGCCGCCCGTCCA...
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
ACCGCCGAGACCGCGTCCGCCCCGCGAGCACAGAGCCTCGCCTTTGCCGATCCGCCGCCCGTCCA...
****Alineamiento****
secuencia: gi|1099288504|ref|XM_019030619.1| PREDICTED: Gorilla gorilla gorilla actin beta (ACTB), mRNA
longitud: 1825
e value: 1.29882e-23
ACCGCCGAGACCGCGTCCGCCCCGCGAGCACAGAGCCTCGCCTTTGCCGATCCGCCGCCCGTCCA...
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
ACCGCCGAGACCGCGTCCGCCCCGCGAGCACAGAGCCTCGCCTTTGCCGATCCGCCGCCCGTCCA...
****Alineamiento****
secuencia: gi|343960005|dbj|AK306863.1| Pan troglodytes mRNA for actin, cytoplasmic 1, complete cds, clone: PtsC-1-5_F07
longitud: 1824
e value: 1.29882e-23
ACCGCCGAGACCGCGTCCGCCCCGCGAGCACAGAGCCTCGCCTTTGCCGATCCGCCGCCCGTCCA...
||||||||||||||||||||||||||||||||||||||||

# `SeqRecord` y `SeqIO`

Frecuentemente,las secuiencias tienen alguna informacion adicional asociadas a ellas. 

Un buen ejemplo fue el ejercicio BLAST que nos dio secuencias asociadas a organismos especificos, y locaciones genomicas.

Generalmente se necesitará guardar esta informacion junto con nuestra secuencia de origen `Seq` .

BioPython permite hacer esto con las clases `SeqRecord` and `SeqIO`.


##  Haciendo Parsing con `SeqIO`

La clase  `SeqIO` que significa: Sequence Input/Output nos permite leer y escribir de diferentes formatos de archivos de anotacion de secuencias que soin comunes en las bases de datos biologicas.

P.Ej. en la base de datos  de secuencias [GenBank](https://www.ncbi.nlm.nih.gov/genbank/) se puede buscar una secuencia relacionada a la plaga de la Peste Bubonica (Yersinia Pestis bacteria) [here](https://www.ncbi.nlm.nih.gov/nuccore/NZ_ADRZ01000932.1?report=fasta).

![](https://www.nationalgeographic.com/content/dam/science/photos/000/033/3338.ngsversion.1492437604403.adapt.676.1.jpg)
("The Triumph of Death by  Pieter Bruegel)

Tendremos un archivo  `fasta` y BioPython puede reconocerlo (hacer parsing) automaticamente.

La anotacion de una secuencia FASTA se encuentra en la cabecera:

```
>NZ_ADRZ01000932.1 Yersinia pestis biovar Antiqua str. E1979001 Contig_E1979001_19275, whole genome shotgun sequence
```

El método `SeqIO.parse` recibe la ruta a un archivo  y su formato, en este caso "fasta" y produce un iteratror sobre cada item en archivo fasta.

Cada item producido por el iterator es un objeto tipo `SeqRecords`.


In [34]:
from Bio import SeqIO

records = SeqIO.parse("plague.fa", "fasta")
for r in records:
    print(type(r))
    print([a for a in dir(r) if not a.startswith("_")])

<class 'Bio.SeqRecord.SeqRecord'>
['annotations', 'dbxrefs', 'description', 'features', 'format', 'id', 'letter_annotations', 'lower', 'name', 'reverse_complement', 'seq', 'translate', 'upper']


### Objetos `SeqRecord` 

Si usted tiene un archivo con solo un registro tipo fasta, puede usar la funcion `SeqIO.read()`.


In [35]:
record = SeqIO.read("single_plague.fa", "fasta")

El objeto `SeqRecords` tiene algunos atributos interesantes.

Contiene un objeto tipo  `Seq`con información de la secuencia.

In [36]:
print(record)

ID: NZ_ADRZ01000932.1
Name: NZ_ADRZ01000932.1
Description: NZ_ADRZ01000932.1 Yersinia pestis biovar Antiqua str. E1979001 Contig_E1979001_19275, whole genome shotgun sequence
Number of features: 0
Seq('CTCTCCCAGCTGAGCTATAGCCCCAATGCGCACATAATAAATCGTGTGAACGGG...AGC', SingleLetterAlphabet())


In [37]:
print(record.seq)

CTCTCCCAGCTGAGCTATAGCCCCAATGCGCACATAATAAATCGTGTGAACGGGGCGCATGATATGAGACCCCCGAAACTGTGTCAACGGCTAAATCGATTTCTCGTGTTAAGCGCTGAAAAAGCGGCCAAATCAGCCTGCAAATAACATAATAAGTGGAATGATGTTCACAAATTTGTTGTCACACCGCTGCTGTTATCAAATATAATAAATATCCTCCGGCATAGC


In [38]:
print(record.id)

NZ_ADRZ01000932.1


In [39]:
print(record.description)

NZ_ADRZ01000932.1 Yersinia pestis biovar Antiqua str. E1979001 Contig_E1979001_19275, whole genome shotgun sequence


Tambien podemos crear nuestros propios objetos tipo `SeqRecord`.

In [40]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
record = SeqRecord(Seq("MKQHKAMIVALIVICITAVVAALVTRKDLCEVHIRTGQTEVAVF",
                       IUPAC.protein),
                   id="YP_025292.1", name="HokC",
                   description="toxic membrane protein, small")
print(record)

ID: YP_025292.1
Name: HokC
Description: toxic membrane protein, small
Number of features: 0
Seq('MKQHKAMIVALIVICITAVVAALVTRKDLCEVHIRTGQTEVAVF', IUPACProtein())


Despues, lo podemos escribir en formato FASTA.

In [36]:
record.format("fasta")

'>YP_025292.1 toxic membrane protein, small\nMKQHKAMIVALIVICITAVVAALVTRKDLCEVHIRTGQTEVAVF\n'

Y despues guardarlo en un archivo. 

Entonces ya sabemos como leer información de secuencias y tambien producir nuestros propios registros de secuencias y guardarlos

In [37]:
with open("myfasta.fa", "w") as f:
    f.write(record.format("fasta"))
    

### Trabajando con formatos  `EMBL ó Genbank`
La mayoria de las veces los archivos GenBank contienen un solo record para un cromosoma o plasmido. asi que usaremos la funcion SeqIO.read(...) sabiendo que el segundo argumento de esta funcion es el formato del archivo...

In [2]:
from Bio import SeqIO
record=SeqIO.read("NC_000913.gbk", "genbank")
print(record.id)
print(len(record))
print(len(a))

IOError: [Errno 2] No such file or directory: 'NC_000913.gbk'

Que son las .features ? -Es una lista Python que contiene un objeto Biopython tipo SeqFeature para 
cada característica (feature) en el archivo GenBank.
Por ejemplo,

In [ ]:
my_gene = record.features[3]
print(my_gene)

Hacer una impresión como esta intenta dar una pantalla legible. Hay tres propiedades clave, .type que es una cadena como el gen o CDS, .location que describe en qué parte del genoma está esta característica, y .qualifiers que es un diccionario de Python lleno de todas las anotaciones de la característica (p.ej. identificadores de genes ).

Asi es como se ve un gene en formato GenBank

gene            337..2799
                /gene="thrA"
                /locus_tag="b0002"
                /gene_synonym="ECK0002; Hs; JW0001; thrA1; thrA2; thrD"
                /db_xref="EcoGene:EG10998"
                /db_xref="GeneID:945803"

### Locaciones de las caracteristicas - Feature Locations
Continuamos con el mismo ejemplo


In [ ]:
print(my_gene.qualifiers["locus_tag"])
print(my_gene.location)
print(my_gene.location.start)
print(my_gene.location.end)
print(my_gene.location.strand)

Recuerde que en el archivo GenBank esta ubicación simple era 337..2799, pero en Biopython se ha convertido en un valor inicial de 336 y 2799 como final. La razón de esto es para que coincida con cómo funciona el conteo de Python, en particular, cómo se corta la cadena de Python. Para extraer esta secuencia del genoma completo necesitamos usar valores de división de 336 y 2799:

Esta era una ubicación muy simple en la hebra principal  si hubiera estado en la hebra inversa necesitaría tomar el complemento inverso. Además, si la ubicación hubiera sido una ubicación compuesta más complicada como una unión (utilizada para genes eucarióticos donde el CDS está formado por varios exones), la ubicación tendría que considerar algunas partes secundarias.

Todas estas complicaciones son atendidas por usted a través del método .extract (...) que toma la secuencia completa del registro padre como un argumento:

In [1]:
gene_seq = my_gene.extract(record.seq)
len(gene_seq)
print(gene_seq)

NameError: name 'my_gene' is not defined

### Ejercicio: 
termine la siguiente secuencia de comandos estableciendo un nombre de caracteristica apropiado como la etiqueta de locus o el número GI (use la información de .qualifiers o .dbxrefs) para extraer todas las secuencias de codificación del archivo GenBank:

In [3]:
## EJERCIO CON LA MODIFIACION SOLICITADA

from Bio import SeqIO
record = SeqIO.read("NC_000913.gbk", "genbank")
output_handle = open("NC_000913_cds.fasta", "w")
count = 0
for feature in record.features:
    if feature.type == "CDS":
        count = count + 1
        feature_name = feature.qualifiers # Use feature.qualifiers or feature.dbxrefs here
        feature_seq = feature.extract(record.seq)
        # Simple FASTA output without line wrapping:
        output_handle.write(">" + str(feature_name) + "\n" + str(feature_seq) + "\n")
output_handle.close()
print(str(count) + " CDS sequences extracted")

4319 CDS sequences extracted


verifique sus secuencias usando el archivo FASTA de NCBI provisto:  NC_000913.ffn
###  Ejercicio: Puede recrear el esquema de nombres de NCBI como se usa en  NC_000913.ffn?

### Longitud de caracteristicas
La longitud de los objetos SeqFeature de Biopython (y los objetos de ubicación) se define como la longitud de la región de secuencia que describen (es decir, cuántas bases están incluidas; o para la anotación de proteínas, cuántos aminoácidos).

In [ ]:
len(my_gene)

Este ejemplo recorre todas las funciones en busca de registros de  genes y calcula su longitud total:

In [2]:
# LA MODIFICACION DE ESTE EJERCICIO SE ENCUENTRA EN LA ULTIMA PARTE

from Bio import SeqIO
record = SeqIO.read("NC_000913.gbk", "genbank")
total = 0
for feature in record.features:
    if feature.type == "gene":
        total = total + len(feature)
print("Total length of all genes is " + str(total))

Total length of all genes is 4137209


### Ejercicio: dar un recuento por separado para cada tipo de caracteristica. Use un diccionario donde los indices son el tipo de caracteristica  (por ejemplo, "gen" y "CDS") y los valores son el recuento para ese tipo.

###Tarea:
4. Modificar el Notebook y Agregar los  ejercicios solicitados.
5. Crear un archivo C471-lab06-Nombre y Apellido.zip que contenga el Notebook modificado, y los archivos creados.

In [1]:
#!/usr/bin/python

from Bio import SeqIO
record = SeqIO.read("NC_000913.gbk", "genbank")
total = 0
#lista para almacenar los tipos de información en la secuencia
types=[]
for feature in record.features:
    feature_name = feature.qualifiers   
    types.append(feature.type)

# eliminamos los datos repetidos de la lista types
types=list(set(types))           

# cantidades de cada tipo
sizes=[0]*len(types)

#diccionario que almacenará los resultados de los tipos y las cantidades
dictionary=dict(zip(types,sizes))


for feature in record.features:
    feature_name = feature.qualifiers   
    dictionary.update({feature.type:dictionary[feature.type]+1})
    
comprobacion=0
print("Imprimiendo tipos encontrados y cantidades por cada uno: ")
for key in dictionary:
    print key, ":", dictionary[key]
    comprobacion=comprobacion+dictionary[key]


Imprimiendo tipos encontrados y cantidades por cada uno: 
misc_feature : 13547
mobile_element : 49
source : 1
rep_origin : 1
tRNA : 89
CDS : 4319
repeat_region : 355
tmRNA : 2
rRNA : 22
STS : 49
gene : 4497
ncRNA : 65
